# Inital Supervised Learning performed on the model

### Imports

In [1]:
import os
import numpy as np # type: ignore
import time
import torch
import torch.nn as nn # type: ignore
import torch.optim as optim # type: ignore
import helper_functions as helper
from torch.utils.data import DataLoader, random_split # type: ignore
from chess import pgn # type: ignore
from tqdm import tqdm # type: ignore
from dataset import ChessPGNDataset
from model import ChessNet, ResBlock

#### Device Agnostic Code

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda


In [3]:
print(torch.__version__)

2.10.0+cu126


# Processing the PGNs

### Creating Move Map

In [4]:
MOVE_MAP = helper.create_move_map()

In [5]:
MOVE_MAP_LENGTH = len(MOVE_MAP)
print(MOVE_MAP_LENGTH)

8192


### Loading the data

In [6]:
pgn_path = "../lichess_db_standard_rated_2026-01.pgn/lichess_db_standard_rated_2026-01.pgn"

In [7]:
pgn_dataset_v1 = ChessPGNDataset(pgn_file_path=pgn_path, move_map=MOVE_MAP)

In [8]:
print(len(pgn_dataset_v1))

7925


In [9]:
total_size = len(pgn_dataset_v1)
train_size = int(0.8 * total_size) 
val_size = total_size - train_size

In [10]:
train_dataset, val_dataset = random_split(pgn_dataset_v1, [train_size, val_size])

In [11]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [ ]:
print(len(next(iter(train_loader)))) 

3


#### Stuff to do before training begins

In [ ]:
train_data_iter = iter(train_loader)
images, labels, games = next(train_data_iter)

print(f"Batch shape (Images): {images.shape}") 
print(f"Batch shape (Labels): {labels.shape}") 
print(f"Batch shape (Labels): {games.shape}")
print(f"First label: {labels[0].item()}")      

Batch shape (Images): torch.Size([64, 12, 8, 8])
Batch shape (Labels): torch.Size([64])
Batch shape (Labels): torch.Size([64, 1])
First label: 1318


In [ ]:
val_data_iter = iter(val_loader)
images, labels, games = next(val_data_iter)

print(f"Batch shape (Images): {images.shape}") 
print(f"Batch shape (Labels): {labels.shape}") 
print(f"Batch shape (Labels): {games.shape}")
print(f"First label: {labels[0].item()}")      

Batch shape (Images): torch.Size([64, 12, 8, 8])
Batch shape (Labels): torch.Size([64])
Batch shape (Labels): torch.Size([64, 1])
First label: 4298


# Making the model and training loop

#### Making the model

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda


In [ ]:
model = ChessNet(MOVE_MAP_LENGTH)
model.to(device)
policy_criterion = nn.CrossEntropyLoss()
value_criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)

#### Making the training loop

In [ ]:
test_batch = next(iter(train_loader))
test_images, test_labels, test_values = [x.to(device) for x in test_batch]
print(f"Test images: min={test_images.min():.4f}, max={test_images.max():.4f}")
print(f"Test values: min={test_values.min():.4f}, max={test_values.max():.4f}, mean={test_values.mean():.4f}")

with torch.no_grad():
    pred_pol, pred_val = model(test_images)
    print(f"Pred values: min={pred_val.min():.4f}, max={pred_val.max():.4f}")
    test_loss = torch.nn.functional.mse_loss(pred_val, test_values.float())
    print(f"Test MSE loss: {test_loss.item():.6f}")


Test images: min=0.0000, max=1.0000
Test values: min=-1.0000, max=1.0000, mean=0.0469
Pred values: min=-0.1894, max=0.2884
Test MSE loss: 0.931998


In [ ]:
print(f"Move map size: {len(MOVE_MAP)}")
print(f"Policy head output size: {MOVE_MAP_LENGTH}")
print(f"Move indices range: 0 to {max(MOVE_MAP.values())}")

out_of_range = [idx for idx in MOVE_MAP.values() if idx >= MOVE_MAP_LENGTH]
if out_of_range:
    print(f"WARNING: {len(out_of_range)} move indices are >= {MOVE_MAP_LENGTH}!")
else:
    print(f"All move indices are within valid range [0, {MOVE_MAP_LENGTH-1}]")

test_batch = next(iter(train_loader))
test_images, test_labels, test_values = test_batch
print(f"\nBatch shapes:")
print(f"  Images: {test_images.shape}")
print(f"  Labels (move indices): {test_labels.shape}, min={test_labels.min()}, max={test_labels.max()}")
print(f"  Values: {test_values.shape}")


Move map size: 8192
Policy head output size: 8192
Move indices range: 0 to 8191
All move indices are within valid range [0, 8191]

Batch shapes:
  Images: torch.Size([64, 12, 8, 8])
  Labels (move indices): torch.Size([64]), min=521, max=8107
  Values: torch.Size([64, 1])


In [ ]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               optimizer: torch.optim.Optimizer,
               policy_criterion,
               value_criterion,
               device: torch.device = device):
        
    running_policy_loss = 0.0
    running_value_loss = 0.0
    model.train()
    model.to(device)

    for batch, (image, move_labels, game_values) in enumerate(data_loader):

        image, move_labels, game_values = image.to(device), move_labels.to(device), game_values.to(device).float()
        
        # Validate inputs
        if move_labels.min() < 0 or move_labels.max() >= MOVE_MAP_LENGTH:
            print(f"WARNING: Invalid move indices in batch {batch}: min={move_labels.min()}, max={move_labels.max()}")
            continue
            
        optimizer.zero_grad()

        pred_policy, pred_value = model(image)
        
        # Check for NaN/Inf
        if torch.isnan(pred_policy).any() or torch.isinf(pred_policy).any():
            print(f"WARNING: NaN/Inf in policy predictions at batch {batch}")
            continue
        if torch.isnan(pred_value).any() or torch.isinf(pred_value).any():
            print(f"WARNING: NaN/Inf in value predictions at batch {batch}")
            continue

        policy_loss = policy_criterion(pred_policy, move_labels)
        value_loss = torch.nn.functional.mse_loss(pred_value, game_values)

        total_loss = policy_loss + value_loss
        total_loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        running_policy_loss += policy_loss.item()
        running_value_loss += value_loss.item()

    print(f"Train policy loss: {running_policy_loss/len(data_loader):.5f} | Train value loss: {running_value_loss/len(data_loader):.5f}\n")


In [20]:
def test_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               optimizer: torch.optim.Optimizer,
               policy_criterion,
               value_criterion,
               device: torch.device = device):
    running_policy_loss = 0.0
    running_value_loss = 0.0
    model.eval()
    model.to(device)

    with torch.inference_mode(): 
        for batch, (image, move_labels, game_values) in enumerate(data_loader):
            image, move_labels, game_values = image.to(device), move_labels.to(device), game_values.to(device).float()
            
            # Validate inputs
            if move_labels.min() < 0 or move_labels.max() >= MOVE_MAP_LENGTH:
                print(f"WARNING: Invalid move indices in batch {batch}: min={move_labels.min()}, max={move_labels.max()}")
                continue
            
            pred_policy, pred_value = model(image)

            policy_loss = policy_criterion(pred_policy, move_labels)
            value_loss = torch.nn.functional.mse_loss(pred_value, game_values)

            running_policy_loss += policy_loss.item()
            running_value_loss += value_loss.item()

            avg_test_policy_loss = running_policy_loss / len(data_loader)
    print(f"Test policy loss: {avg_test_policy_loss:.5f} | Test value loss: {running_value_loss/len(data_loader):.5f}\n")

    return avg_test_policy_loss


In [21]:
from pathlib import Path

best_test_loss = float('inf')

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, 
                 exist_ok=True 
)

MODEL_NAME = "supervised_learning_chess_model_1.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME
epochs = 200
for epoch in tqdm(range(epochs)):
    tqdm.write(f"Epoch: {epoch}\n---------")
    train_step(model,
               train_loader,
               optimizer,
               policy_criterion,
               value_criterion,
               device
               )
    current_test_loss = test_step(model,
               val_loader,
               optimizer,
               policy_criterion,
               value_criterion,
               device
               )
    scheduler.step(current_test_loss)
    if current_test_loss < best_test_loss:
        best_test_loss = current_test_loss
        print(f"⭐ New Best Model! Loss improved to: {best_test_loss:.5f}. Saving...")
        torch.save(obj=model.state_dict(), f=MODEL_SAVE_PATH)
    else:
        print(f"No improvement in Test Loss ({current_test_loss:.5f} vs best {best_test_loss:.5f}).")

    current_lr = optimizer.param_groups[0]['lr']
    print(f"Current Learning Rate: {current_lr}")


  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 0
---------
Train policy loss: 6.38006 | Train value loss: 0.97327

Test policy loss: 5.83408 | Test value loss: 0.97458

⭐ New Best Model! Loss improved to: 5.83408. Saving...


  0%|          | 1/200 [00:04<13:35,  4.10s/it]

Current Learning Rate: 0.001
Epoch: 1
---------
Train policy loss: 4.75792 | Train value loss: 0.95804

Test policy loss: 5.48632 | Test value loss: 0.95798

⭐ New Best Model! Loss improved to: 5.48632. Saving...


  1%|          | 2/200 [00:07<12:58,  3.93s/it]

...

Test policy loss: 5.97959 | Test value loss: 1.07263

No improvement in Test Loss (5.97959 vs best 5.29527).
Current Learning Rate: 1.0000000000000004e-08
Epoch: 198
---------
Train policy loss: 0.20860 | Train value loss: 0.09339



100%|█████████▉| 199/200 [12:32<00:03,  3.87s/it]

Test policy loss: 5.98638 | Test value loss: 1.07651

No improvement in Test Loss (5.98638 vs best 5.29527).
Current Learning Rate: 1.0000000000000004e-08
Epoch: 199
---------
Train policy loss: 0.21549 | Train value loss: 0.09409



100%|██████████| 200/200 [12:36<00:00,  3.78s/it]

Test policy loss: 6.00758 | Test value loss: 1.07568

No improvement in Test Loss (6.00758 vs best 5.29527).
Current Learning Rate: 1.0000000000000004e-08
